In [3]:
# Import necessary packages
import pandas as pd
import numpy as np

import sys

In [4]:
def read_file(file_name):
    """
    Creates an adjacency matrix from the csv file
    """
    df = pd.read_csv(file_name)
    
    # Dictionary to store cities' indices
    vertices_id = {}
    cnt = 0
    
    # Iterate all rows and assign an unique id to each city
    for _, row in df.iterrows():   
        for city in [row[0], row[1]]:
            if city not in vertices_id:
                vertices_id[city] = cnt
                cnt += 1
    num_cities = len(vertices_id)
    
    # Create an adjacency matrix of shape (num_vertices, num_vertices)
    adj_matrix = np.zeros((num_cities, num_cities))
    # Iterate the rows again to build the adj matrix
    for _, row in df.iterrows():
        idx_1 = vertices_id[row[0]]
        idx_2 = vertices_id[row[1]]
        # Add the connection to adjacency matrix (both directions)
        adj_matrix[idx_1][idx_2] = row[2]
        adj_matrix[idx_2][idx_1] = row[2]
        
    return adj_matrix, vertices_id

In [8]:
class Graph:
    def __init__(self, adj_matrix, vertices_id):
        self.adj_matrix = adj_matrix
        self.vertices_id = vertices_id
        self.id_vertices = {v: k for k, v in vertices_id.items()}
        
    def neighbors(self, v):
        """
        Returns indices of v's neighbors
        """
        row = adj_matrix[v]
        return np.where(row != 0)[0]
        
    def dfs_helper(self, current, visited_list, parent):
        # Scan through the neighbors of the current vertex
        for neighbor in self.neighbors(current):
            # If it is not visited, visit it
            if neighbor not in visited_list:
                # Mark this vertex as visited
                visited_list.append(neighbor)
                # We visit this vertex from "current", so mark it as the parent 
                parent[neighbor] = current
                # Recur!
                self.dfs_helper(neighbor, visited_list, parent)
        
    def dfs(self, start):
        """
        Perform DFS from the given starting vertex
        """
        # Store parent information
        parent = {start: None}
        # Keep track of already visited vertices
        visited_list = [start]
        
        # Call the recursive helper function, real work starts here
        self.dfs_helper(start, visited_list, parent)
        return visited_list, parent
        
    def degree(self, vertex_name):
        """
        Returns the degree of a given vertex
        """
        id = self.vertices_id[vertex_name]
        return len(self.neighbors(id))
    
    def components(self):
        """
        Returns the number of connected components in the graph
        """
        big_visited_list = []
        component_count = 0
        
        # Perform DFS on every vertices, one completion of a DFS corresponds to one connnected component
        for i in range(self.adj_matrix.shape[0]):
            if i not in big_visited_list:
                visited_list, _ = self.dfs(i)
                big_visited_list.extend(visited_list)
                component_count += 1
        return component_count
    
    def path(self, vertex_name_1, vertex_name_2):
        """
        Returns the path between two vertices if it exists
        """
        vert_idx_1 = self.vertices_id[vertex_name_1]
        vert_idx_2 = self.vertices_id[vertex_name_2]
        # Perform DFS from vertex 1 to see if there's a path to vertex 2
        visited_list, parent = self.dfs(vert_idx_1)
        
        # If it exists, backtrack the path from vertex 2 using information stored in parent
        if vert_idx_2 in visited_list:
            current = vert_idx_2
            p = []
            while (current is not None):
                p.append(current)
                current = parent[current]
            p.reverse()
            return [self.id_vertices[id] for id in p] 
        else:
            return None


adj_matrix, vertices_id = read_file("cities.csv")
cities_graph = Graph(adj_matrix, vertices_id)

In [9]:
cities_graph.degree("Ha Noi")

5

In [10]:
cities_graph.components()

17

In [8]:
cities_graph.path("Ha Noi", "Can Tho")

['Ha Noi',
 'Vinh',
 'Dong Hoi',
 'Hue',
 'Da Nang',
 'Hoi An',
 'Phuoc Son',
 'Kontum',
 'Buon Ma Thuot',
 'Ho Chi Minh City',
 'Can Tho']